In [1]:
from tmilib import *

In [2]:
from reconstruct_focus_times_common import *
from reconstruct_focus_times import ReconstructFocusTimesBaseline
from session_tracker import get_focused_tab

In [3]:
'''
for user in list_users():
  ordered_visits = get_history_ordered_visits_for_user(user)
  if len(ordered_visits) == 0:
    continue
  last_visit = ordered_visits[-1]
  last_visit_time = float(last_visit['visitTime']) / 1000.0
  print user, datetime.datetime.fromtimestamp(last_visit_time)
'''

"\nfor user in list_users():\n  ordered_visits = get_history_ordered_visits_for_user(user)\n  if len(ordered_visits) == 0:\n    continue\n  last_visit = ordered_visits[-1]\n  last_visit_time = float(last_visit['visitTime']) / 1000.0\n  print user, datetime.datetime.fromtimestamp(last_visit_time)\n"

In [4]:
'''
for user in list_users():
  tab_focus_times = get_tab_focus_times_for_user(user)
  if len(tab_focus_times) == 0:
    continue
  last_visit = tab_focus_times[-1]
  last_visit_time = float(last_visit['end']) / 1000.0
  print user, datetime.datetime.fromtimestamp(last_visit_time)
'''

"\nfor user in list_users():\n  tab_focus_times = get_tab_focus_times_for_user(user)\n  if len(tab_focus_times) == 0:\n    continue\n  last_visit = tab_focus_times[-1]\n  last_visit_time = float(last_visit['end']) / 1000.0\n  print user, datetime.datetime.fromtimestamp(last_visit_time)\n"

In [5]:
def ignore_all_before_start_or_after_end(visit_lengths, start_time, end_time):
  output = []
  for x in visit_lengths:
    if x['end'] < start_time:
      continue
    if x['start'] > end_time:
      continue
    #if x['start'] < start_time: # - 1000: # 1 second before or after
    #  continue
    #if x['end'] > end_time: # + 1000:
    #  continue
    output.append(x)
  return output

In [6]:
def get_evaluated_reference_and_reconstructed_focus_times_for_user(user):
  #tab_focus_times = get_tab_focus_times_for_user(user)
  tab_focus_times = get_tab_focus_times_only_tab_updated_for_user(user)
  reconstructed_tab_focus_times = list(get_reconstruct_focus_times_baseline_for_user(user))
  if len(tab_focus_times) == 0 or len(reconstructed_tab_focus_times) == 0:
    return [],[]
  ref_start_time = max(get_earliest_start_time(tab_focus_times), get_earliest_start_time(reconstructed_tab_focus_times))
  ref_end_time = min(get_last_end_time(tab_focus_times), get_last_end_time(reconstructed_tab_focus_times))
  evaluated_reconstructed_tab_focus_times = ignore_all_before_start_or_after_end(reconstructed_tab_focus_times, ref_start_time, ref_end_time)
  evaluated_tab_focus_times = ignore_all_before_start_or_after_end(tab_focus_times, ref_start_time, ref_end_time)
  return evaluated_tab_focus_times,evaluated_reconstructed_tab_focus_times

def get_evaluation_stats_for_user(user):
  evaluated_tab_focus_times,evaluated_reconstructed_tab_focus_times = get_evaluated_reference_and_reconstructed_focus_times_for_user(user)
  if len(evaluated_reconstructed_tab_focus_times) == 0 or len(evaluated_tab_focus_times) == 0:
    return {}
  return evalutate_tab_focus_reconstruction(evaluated_tab_focus_times, evaluated_reconstructed_tab_focus_times)

'''
def get_evaluation_stats_for_user(user):
  #evaluated_tab_focus_times,evaluated_reconstructed_tab_focus_times = get_evaluated_reference_and_reconstructed_focus_times_for_user(user)
  #if len(evaluated_reconstructed_tab_focus_times) == 0 or len(evaluated_tab_focus_times) == 0:
  #  return {}
  tab_focus_times = get_tab_focus_times_for_user(user)
  reconstructed_tab_focus_times = list(get_reconstruct_focus_times_baseline_for_user(user))
  if len(tab_focus_times) == 0 or len(reconstructed_tab_focus_times) == 0:
    return {}
  ref_start_time = max(get_earliest_start_time(tab_focus_times), get_earliest_start_time(reconstructed_tab_focus_times))
  ref_end_time = min(get_last_end_time(tab_focus_times), get_last_end_time(reconstructed_tab_focus_times))
  evaluated_reconstructed_tab_focus_times = ignore_all_before_start_or_after_end(reconstructed_tab_focus_times, ref_start_time, ref_end_time)
  evaluated_tab_focus_times = ignore_all_before_start_or_after_end(tab_focus_times, ref_start_time, ref_end_time)
  result_reference = second_to_active_url(evaluated_tab_focus_times, ref_start_time, ref_end_time)
  result_reconstructed = second_to_active_url(evaluated_reconstructed_tab_focus_times, ref_start_time, ref_end_time)
  return evaluation_stats_for_reconstruction(result_reference, result_reconstructed)
  #print tab_focus_times[0].keys()
  #print list(reconstructed_tab_focus_times)[0].keys()
'''

'\ndef get_evaluation_stats_for_user(user):\n  #evaluated_tab_focus_times,evaluated_reconstructed_tab_focus_times = get_evaluated_reference_and_reconstructed_focus_times_for_user(user)\n  #if len(evaluated_reconstructed_tab_focus_times) == 0 or len(evaluated_tab_focus_times) == 0:\n  #  return {}\n  tab_focus_times = get_tab_focus_times_for_user(user)\n  reconstructed_tab_focus_times = list(get_reconstruct_focus_times_baseline_for_user(user))\n  if len(tab_focus_times) == 0 or len(reconstructed_tab_focus_times) == 0:\n    return {}\n  ref_start_time = max(get_earliest_start_time(tab_focus_times), get_earliest_start_time(reconstructed_tab_focus_times))\n  ref_end_time = min(get_last_end_time(tab_focus_times), get_last_end_time(reconstructed_tab_focus_times))\n  evaluated_reconstructed_tab_focus_times = ignore_all_before_start_or_after_end(reconstructed_tab_focus_times, ref_start_time, ref_end_time)\n  evaluated_tab_focus_times = ignore_all_before_start_or_after_end(tab_focus_times, ref_

In [7]:
class ClosestVisitTimeForUrl:
  def __init__(self, tab_focus_times):
    self.visits = tab_focus_times
    self.url_to_visit_times = {} # url to start of visit
    self.visit_time_to_visit = {}
    for visit in self.visits: # these visits are time-ordered
      start = visit['start']
      url = visit['url']
      if url not in self.url_to_visit_times:
        self.url_to_visit_times[url] = []
      self.url_to_visit_times[url].append(start)
      self.visit_time_to_visit[start] = visit
  def get_visit_from_time(self, target_visit_time):
    self.visit_time_to_visit.get(target_visit_time, None)
  def find_closest_visit(self, url, target_visit_time):
    if url not in self.url_to_visit_times:
      return None
    all_visits = self.url_to_visit_times[url]
    closest_visit = None
    for visit in all_visits:
      if closest_visit == None:
        closest_visit = visit
        continue
      if abs(visit - target_visit_time) < abs(closest_visit - target_visit_time):
        closest_visit = visit
    return closest_visit

In [8]:
def get_span_alignment_stats_for_user(user):
  evaluated_tab_focus_times,evaluated_reconstructed_tab_focus_times = get_evaluated_reference_and_reconstructed_focus_times_for_user(user)
  if len(evaluated_reconstructed_tab_focus_times) == 0 or len(evaluated_tab_focus_times) == 0:
    return {}
  closest_visit_times_for_reference = ClosestVisitTimeForUrl(evaluated_tab_focus_times)
  closest_visit_times_for_reconstructed = ClosestVisitTimeForUrl(evaluated_reconstructed_tab_focus_times)
  stats = Counter()
  domains_where_reconstructed_has_no_reference = Counter()
  domains_where_reference_has_no_reconstructed = Counter()
  urls_where_reconstructed_has_no_reference = Counter()
  urls_where_reference_has_no_reconstructed = Counter()
  reconstructed_transition_types_with_no_corresponding_reference = Counter()
  reconstructed_transition_types_total = Counter()
  reconstructed_visits_labeled = []
  reference_visits_labeled = []
  for visit in evaluated_reconstructed_tab_focus_times:
    start_time = visit['start']
    url = visit['url']
    domain = url_to_domain(url)
    closest_time = closest_visit_times_for_reference.find_closest_visit(url, start_time)
    visit_labeled = {k:v for k,v in visit.items()}
    if 'transition' in visit:
      transition = visit['transition']
      reconstructed_transition_types_total[transition] += 1
    if closest_time == None or abs(start_time - closest_time) > 5*1000: # more than 5 seconds off
      stats['reconstructed_has_no_corresponding_reference'] += 1
      domains_where_reconstructed_has_no_reference[domain] += 1
      urls_where_reconstructed_has_no_reference[url] += 1
      visit_labeled['have_reference'] = False
      if 'transition' in visit:
        transition = visit['transition']
        reconstructed_transition_types_with_no_corresponding_reference[transition] += 1
    else:
      stats['reconstructed_has_corresponding_reference'] += 1
      visit_labeled['have_reference'] = True
    reconstructed_visits_labeled.append(visit_labeled)
  for visit in evaluated_tab_focus_times:
    start_time = visit['start']
    url = visit['url']
    closest_time = closest_visit_times_for_reconstructed.find_closest_visit(url, start_time)
    visit_labeled = {k:v for k,v in visit.items()}
    if closest_time == None or abs(start_time - closest_time) > 5*1000: # more than 5 seconds off
      stats['reference_has_no_corresponding_reconstructed'] += 1
      domains_where_reference_has_no_reconstructed[domain] += 1
      urls_where_reference_has_no_reconstructed[url] += 1
      visit_labeled['have_reconstructed'] = False
    else:
      stats['reference_has_corresponding_reconstructed'] += 1
      visit_labeled['have_reconstructed'] = True
    reference_visits_labeled.append(visit_labeled)
  return {
    'stats': stats,
    'domains_where_reconstructed_has_no_reference': domains_where_reconstructed_has_no_reference,
    'domains_where_reference_has_no_reconstructed': domains_where_reference_has_no_reconstructed,
    'urls_where_reconstructed_has_no_reference': urls_where_reconstructed_has_no_reference,
    'urls_where_reference_has_no_reconstructed': urls_where_reference_has_no_reconstructed,
    'reconstructed_visits_labeled': reconstructed_visits_labeled,
    'reference_visits_labeled': reference_visits_labeled,
    'reconstructed_transition_types_with_no_corresponding_reference': reconstructed_transition_types_with_no_corresponding_reference,
    'reconstructed_transition_types_total': reconstructed_transition_types_total,
  }

In [9]:
user_to_span_alignment_data = {}

for user in list_users():
  data = get_span_alignment_stats_for_user(user)
  if 'stats' not in data:
    continue
  user_to_span_alignment_data[user] = data


In [10]:
overall_span_alignment_stats = Counter()
for user,data in user_to_span_alignment_data.items():
  stats = data['stats']
  for k,v in stats.items():
    overall_span_alignment_stats[k] += v

print_counter(overall_span_alignment_stats)
# 60% of reference have corresponding reconstructed
# 84% of reconstructed have corresponding reference

reference_has_corresponding_reconstructed 691955
reconstructed_has_corresponding_reference 689770
reference_has_no_corresponding_reconstructed 255078
reconstructed_has_no_corresponding_reference 179818


In [11]:
overall_transitions_with_no_reference = Counter()

for user,data in user_to_span_alignment_data.items():
  for k,v in data['reconstructed_transition_types_with_no_corresponding_reference'].items():
    overall_transitions_with_no_reference[k] += v

print_counter(overall_transitions_with_no_reference)

link 137553
form_submit 18597
reload 8651
typed 6056
generated 4422
auto_bookmark 3135
auto_toplevel 1359
keyword 40
keyword_generated 5


In [12]:
overall_transitions_total_in_evaluated_reconstruction = Counter()

for user,data in user_to_span_alignment_data.items():
  for k,v in data['reconstructed_transition_types_total'].items():
    overall_transitions_total_in_evaluated_reconstruction[k] += v

print_counter(overall_transitions_total_in_evaluated_reconstruction)


link 669737
form_submit 108915
reload 31720
typed 22748
auto_bookmark 17769
generated 14964
auto_toplevel 3470
keyword 259
keyword_generated 6


In [13]:
# history transition types that have highest levels of no-corresponding-reference
for k,v in overall_transitions_total_in_evaluated_reconstruction.items():
  bad_transitions = overall_transitions_with_no_reference.get(k, 0)
  print k, (float(bad_transitions)/v)

form_submit 0.170747830877
auto_bookmark 0.176430862738
keyword 0.15444015444
typed 0.26622120626
auto_toplevel 0.391642651297
reload 0.272730138714
generated 0.295509222133
link 0.205383605803
keyword_generated 0.833333333333


In [14]:
overall_transitions_total_in_reconstruction = Counter()

for user in list_users_with_hist():
  for visit in get_reconstruct_focus_times_baseline_for_user(user):
    transition = visit['transition']
    overall_transitions_total_in_reconstruction[transition] += 1

print_counter(overall_transitions_total_in_reconstruction)

link 4996461
form_submit 767306
reload 269928
typed 256299
auto_bookmark 135748
generated 131291
auto_toplevel 99731
keyword 2170
keyword_generated 20


In [15]:
overall_transitions_total = Counter()

for user in list_users_with_hist():
  for visit in get_history_ordered_visits_for_user(user):
    transition = visit['transition']
    overall_transitions_total[transition] += 1

print_counter(overall_transitions_total)

link 6157728
form_submit 1099071
reload 412746
typed 265271
auto_bookmark 140369
manual_subframe 137829
generated 131656
auto_toplevel 100801
keyword 2173
auto_subframe 302
keyword_generated 21


In [16]:
for k,v in overall_transitions_total.items():
  bad_transitions = overall_transitions_with_no_reference.get(k, 0)
  print k, (float(bad_transitions)/v)

form_submit 0.0169206538977
auto_subframe 0.0
auto_bookmark 0.0223339911234
keyword 0.0184077312471
typed 0.0228294838109
auto_toplevel 0.0134820091071
reload 0.0209596216559
generated 0.0335875311418
link 0.0223382715183
manual_subframe 0.0
keyword_generated 0.238095238095


In [17]:
for k,v in overall_transitions_total_in_reconstruction.items():
  bad_transitions = overall_transitions_with_no_reference.get(k, 0)
  print k, (float(bad_transitions)/v)



form_submit 0.0242367451838
auto_bookmark 0.0230942628989
keyword 0.0184331797235
typed 0.0236286524723
auto_toplevel 0.0136266557038
reload 0.0320492872173
generated 0.0336809072975
link 0.0275300857947
keyword_generated 0.25


In [18]:
optimal_reconstruction_stats_spans = Counter()
optimal_reconstruction_stats_times = Counter()

for user,data in user_to_span_alignment_data.items():
  for visit in data['reconstructed_visits_labeled']:
    if visit['have_reference']:
      optimal_reconstruction_stats_spans['reconstructed_has_reference'] += 1
    else:
      optimal_reconstruction_stats_spans['reconstructed_has_no_reference'] += 1
  for visit in data['reference_visits_labeled']:
    length = visit['end'] - visit['start']
    if visit['have_reconstructed']:
      optimal_reconstruction_stats_spans['reference_has_reconstructed'] += 1
      optimal_reconstruction_stats_times['seconds_of_reference_that_can_be_reconstructed'] += length
    else:
      optimal_reconstruction_stats_spans['reference_has_no_reconstructed'] += 1
      optimal_reconstruction_stats_times['seconds_of_reference_that_cannot_be_reconstructed'] += length

In [19]:
print_counter(optimal_reconstruction_stats_spans)
# reference has reconstructed 60% = 80332.0 / (80332 + 54018)
# reconstructed has reference 84% = 55748.0 / (10948 + 55748)

# reference has reconstructed 73% = 574617.0/(574617+207343)
# reconstructed has reference 79% = 572765.0/(572765 + 150460)

reference_has_reconstructed 691955
reconstructed_has_reference 689770
reference_has_no_reconstructed 255078
reconstructed_has_no_reference 179818


In [20]:
print_counter(optimal_reconstruction_stats_times)
# seconds of reference that can be reconstructed 57% = 1254885375.005127 / (1254885375.005127 + 947693694.9926758)

seconds_of_reference_that_can_be_reconstructed 13223793000.0
seconds_of_reference_that_cannot_be_reconstructed 7829529826.03


In [21]:
domain_to_seconds_that_cannot_be_reconstructed = Counter()
url_to_seconds_that_cannot_be_reconstructed = Counter()
domain_to_spans_that_cannot_be_reconstructed = Counter()
url_to_spans_that_cannot_be_reconstructed = Counter()

for user,data in user_to_span_alignment_data.items():
  for visit in data['reference_visits_labeled']:
    length = visit['end'] - visit['start']
    url = visit['url']
    domain = url_to_domain(url)
    if not visit['have_reconstructed']:
      domain_to_seconds_that_cannot_be_reconstructed[domain] += length
      domain_to_spans_that_cannot_be_reconstructed[domain] += 1
      url_to_seconds_that_cannot_be_reconstructed[url] += length
      url_to_spans_that_cannot_be_reconstructed[url] += 1

In [22]:
print_counter(domain_to_spans_that_cannot_be_reconstructed)

www.mturk.com 81899
newtab 27145
www.facebook.com 13883
www.youtube.com 10291
mail.google.com 6790
www.google.com 6371
www.reddit.com 5056
www.netflix.com 3601
hitgrabber.net 2871
www.mturkgrind.com 2196
docs.google.com 2057
apps.facebook.com 1882
www.amazon.com 1623
www.twitch.tv 1374
mturkforum.com 1122
us20.chatzy.com 1033
www.tumblr.com 975
play.pokemonshowdown.com 906
www.classmates.com 892
game.soul-arena.com 801
perk.tv 795
us-mg5.mail.yahoo.com 766
www.hulu.com 740
www.trulia.com 671
klbibkeccnjlkjkiokjodocebajanakg 666
www.swagbucks.com 647
 587
www.google.co.in 574
drive.google.com 520
www.mturkcrowd.com 517
www.buzzfeed.com 508
compstak.slack.com 508
www.istreamitall.com 496
twitter.com 474
turkernation.com 462
boards.4chan.org 459
www.surveymonkey.com 454
allmyvideos.net 442
bookmarks 435
www.linkedin.com 425
web.whatsapp.com 420
forums.animez.to 415
www.tohosting.tk 411
www.livenewschat.eu 398
www.host2inger.com 388
www.lipstickalley.com 380
ma.brazzers.com 372
turkopticon

In [23]:
print_counter(domain_to_seconds_that_cannot_be_reconstructed)

www.mturk.com 1660942230.02
www.facebook.com 732779693.99
www.youtube.com 455382050.005
mail.google.com 279977816.996
www.netflix.com 275934071.999
newtab 237388639.005
apps.facebook.com 177301844.0
docs.google.com 173010037.999
www.reddit.com 140525833.001
quarks.inr.ac.ru 75898913.0002
www.google.com 62769052.0
www.amazon.com 62448775.9958
hitgrabber.net 59842238.0076
www.mturkgrind.com 52758396.0024
www.tumblr.com 49766148.9995
plinga.com 49169672.9983
perk.tv 48031874.0002
www.twitch.tv 48000666.0015
allmyvideos.net 38421286.9998
mturkforum.com 35962323.9988
play.pokemonshowdown.com 35860241.0022
us-mg5.mail.yahoo.com 35726392.0012
www.hulu.com 29808123.9983
www.surveymonkey.com 29509142.0032
tweetdeck.twitter.com 28709408.0007
www.cntraveler.com 28216343.9998
www.eatthis.com 27800065.0
www.speechpad.com 27137774.9995
www.livenewschat.eu 26706149.9983
us20.chatzy.com 25348916.0042
www.messenger.com 25216759.0005
game.soul-arena.com 24793837.9995
outlook.live.com 23883988.9993
www.e

In [24]:
print_counter(url_to_spans_that_cannot_be_reconstructed)

chrome://newtab/ 27143
https://www.mturk.com/mturk/dashboard 9722
https://www.mturk.com/mturk/findhits?match=true?hit_scraper 9350
https://www.facebook.com/ 7633
https://www.mturk.com/mturk/findhits?match=false&hit_scraper 4375
http://hitgrabber.net/ 2871
https://www.google.com/webhp?sourceid=chrome-instant&ion=1&espv=2&ie=UTF-8 2259
https://mail.google.com/mail/u/0/#inbox 1883
https://www.mturk.com/mturk/myhits 1348
https://us20.chatzy.com/11920092594925 1033
https://www.mturk.com/HM 964
https://www.mturk.com/mturk/findhits?match=true?hit_scraper-dev 867
http://game.soul-arena.com/ 801
https://www.facebook.com/messages/jessica.wood.106 661
https://www.tumblr.com/dashboard 653
https://www.mturk.com/mturk/findhits?match=true 555
https://www.reddit.com/ 496
https://perk.tv/video/248/perk-app-trailers 490
https://www.mturk.com/mturk/findhits?match=false 488
https://www.mturk.com/mturk/submit 445
https://web.whatsapp.com/ 420
https://mail.google.com/mail/u/0/#spam 378
https://www.mturk.com

In [25]:
print_counter(url_to_seconds_that_cannot_be_reconstructed)

https://www.facebook.com/ 453541301.993
https://www.mturk.com/mturk/dashboard 343325845.007
https://www.mturk.com/mturk/findhits?match=true?hit_scraper 280295818.005
chrome://newtab/ 237313717.005
https://www.mturk.com/mturk/findhits?match=false&hit_scraper 126910716.007
https://docs.google.com/spreadsheets/d/1ZBzbhT5itY-L75wfjrNVJQt177kWhXh72bCTqCfMQyE/edit#gid=2096979058 112991767.0
https://mail.google.com/mail/u/0/#inbox 105126537.998
http://quarks.inr.ac.ru/2004/proceedings/Quarks/kuraev.pdf 75898913.0002
http://hitgrabber.net/ 59842238.0076
https://apps.facebook.com/eontictactoe/?fb_source=bookmark&ref=bookmarks&count=0&fb_bmpos=_0 49315522.0015
https://mail.google.com/mail/u/1/#inbox 45351800.9973
https://www.facebook.com/messages/jessica.wood.106 42008757.0012
https://www.mturk.com/mturk/findhits?match=true?hit_scraper-dev 41693419.0012
https://perk.tv/video/248/perk-app-trailers 40099986.9983
https://www.tumblr.com/dashboard 38489414.9983
https://mail.google.com/mail/u/0/#spam 

In [26]:
overall_domains_where_reconstructed_has_no_reference = Counter()
overall_domains_where_reference_has_no_reconstructed = Counter()
overall_urls_where_reconstructed_has_no_reference = Counter()
overall_urls_where_reference_has_no_reconstructed = Counter()

for user,data in user_to_span_alignment_data.items():
  for k,v in data['domains_where_reconstructed_has_no_reference'].items():
    overall_domains_where_reconstructed_has_no_reference[k] += v
  for k,v in data['domains_where_reference_has_no_reconstructed'].items():
    overall_domains_where_reference_has_no_reconstructed[k] += v
  for k,v in data['urls_where_reconstructed_has_no_reference'].items():
    overall_urls_where_reconstructed_has_no_reference[k] += v
  for k,v in data['urls_where_reference_has_no_reconstructed'].items():
    overall_urls_where_reference_has_no_reconstructed[k] += v


In [27]:
print_counter(overall_urls_where_reconstructed_has_no_reference)

https://www.mturk.com/mturk/dashboard 3674
https://www.google.com/webhp?sourceid=chrome-instant&ion=1&espv=2&ie=UTF-8 3169
https://www.mturk.com/mturk/previewandaccept?groupId=3SFJXQ9EG58Y3WRLXCW2QCVUYYDXDX 1862
https://www.mturk.com/mturk/checkregistration 1775
https://www.facebook.com/ 1638
https://www.mturk.com/mturk/myhits 1599
https://www.mturk.com/mturk/previewandaccept?groupId=35DGDVUOHKKVOAMD7VZRLQEQ0J7YPA 1215
https://www.mturk.com/mturk/previewandaccept?groupId=3NYGPW9OUZ9VT0WI3H1AZXNS6BEZP1 1211
https://perk.tv/video/6268/perk-app-trailers 1129
https://www.mturk.com/mturk/sortmyhits?searchSpec=HITSearch%23T%231%2325%23-1%23T%23%21Status%210%21rO0ABXQACEFzc2lnbmVk%21%23%21Deadline%211%21%23%21 954
https://www.mturk.com/mturk/previewandaccept?groupId=3PBTVBPQ9UI5R0A3PW7WXNE8W4CLGC 870
https://mail.google.com/mail/u/0/#inbox 858
https://www.mturk.com/mturk/findhits?match=false 824
https://www.youtube.com/ 592
https://perk.tv/video/248/perk-app-trailers 585
https://www.mturk.com

In [28]:
print_counter(overall_urls_where_reference_has_no_reconstructed)

chrome://newtab/ 27143
https://www.mturk.com/mturk/dashboard 9722
https://www.mturk.com/mturk/findhits?match=true?hit_scraper 9350
https://www.facebook.com/ 7633
https://www.mturk.com/mturk/findhits?match=false&hit_scraper 4375
http://hitgrabber.net/ 2871
https://www.google.com/webhp?sourceid=chrome-instant&ion=1&espv=2&ie=UTF-8 2259
https://mail.google.com/mail/u/0/#inbox 1883
https://www.mturk.com/mturk/myhits 1348
https://us20.chatzy.com/11920092594925 1033
https://www.mturk.com/HM 964
https://www.mturk.com/mturk/findhits?match=true?hit_scraper-dev 867
http://game.soul-arena.com/ 801
https://www.facebook.com/messages/jessica.wood.106 661
https://www.tumblr.com/dashboard 653
https://www.mturk.com/mturk/findhits?match=true 555
https://www.reddit.com/ 496
https://perk.tv/video/248/perk-app-trailers 490
https://www.mturk.com/mturk/findhits?match=false 488
https://www.mturk.com/mturk/submit 445
https://web.whatsapp.com/ 420
https://mail.google.com/mail/u/0/#spam 378
https://www.mturk.com

In [29]:
print_counter(overall_domains_where_reconstructed_has_no_reference)

www.mturk.com 53749
www.google.com 12160
www.youtube.com 8723
www.facebook.com 5621
www.reddit.com 4451
i.imgur.com 3616
docs.google.com 2839
perk.tv 2779
mail.google.com 2587
imgur.com 2454
www.amazon.com 1934
www.mturkgrind.com 1813
www.host2inger.com 1346
earnhoney.com 1274
www.tohosting.tk 1196
mturkforum.com 1187
i.ytimg.com 1013
mail.aol.com 935
turkopticon.ucsd.edu 897
vitals-2.crowdcomputingsystems.com 851
work.crowdsurfwork.com 802
local.cs-int-592.com:5550 690
www.google.co.in 662
www.surveymonkey.com 659
www.netflix.com 657
turkernation.com 596
www.ebay.com 529
www.hulu.com 526
www.pinterest.com 519
www.bing.com 483
www.buzzfeed.com 480
payments.amazon.com 469
www.yelp.com 441
www.instagram.com 415
personalityfactors.net 402
twitter.com 386
www.lipstickalley.com 384
accounts.google.com 382
hw.reddit.com 368
thewatchseries.to 354
en.wikipedia.org 351
www.swagbucks.com 346
www.car-insurance-quote-fx.com 343
www.twitch.tv 340
www.yahoo.com 337
www.tumblr.com 310
ma.brazzers.com

In [30]:
print_counter(overall_domains_where_reference_has_no_reconstructed)

www.mturk.com 53813
www.facebook.com 27190
turkopticon.ucsd.edu 8090
www.amazon.com 7694
news.google.com 7587
my.presonus.com 6958
www.google.com 6787
www.nextmuni.com 6607
aurafc.guildwork.com 6471
www.mturkcrowd.com 6202
www.reddit.com 6012
www.mturkgrind.com 5883
prolific.ac 5327
www.youtube.com 4742
mail.google.com 4375
vidzi.tv 3977
www.netflix.com 3752
secure.bankofamerica.com 3540
us-mg5.mail.yahoo.com 3483
www.buzzfeed.com 2684
armycivilianservice.usajobs.gov 2674
www.hulu.com 2562
perk.tv 2353
docs.python.org 2319
kat.cr 2313
mintvine.com 2286
gate.aon.com 2234
www.txlottery.org 2135
www.tumblr.com 2104
sewg.az1.qualtrics.com 2029
www.playstationtrophies.org 1738
68.45.57.129:22362 1696
www.insightexpress.com 1634
qbo.intuit.com 1535
www.kayak.com 1533
www.startribune.com 1532
stonybrookuniversity.co1.qualtrics.com 1373
klbibkeccnjlkjkiokjodocebajanakg 1371
www.yahoo.com 1346
www.michigantrafficcrashfacts.org 1322
www.simpsonsworld.com 1313
9gag.com 1296
www.toonova.com 1243
m

In [31]:
user_to_evaluation_stats = {}

for user in list_users():
  try:
    stats = get_evaluation_stats_for_user(user)
  except:
    continue
  user_to_evaluation_stats[user] = stats


In [32]:
overall_evaluation_stats = Counter()
for user,stats in user_to_evaluation_stats.items():
  for k,v in stats.items():
    overall_evaluation_stats[k] += v

print overall_evaluation_stats

Counter({'both_inactive': 164569069, 'correct_url': 12104675, 'ref_active_but_rec_inactive': 6309352, 'incorrect_domain': 1960359, 'ref_inactive_but_rec_active': 1719712, 'correct_domain': 804903})


In [33]:
'''
def get_users_with_tab_focus_data():
  users_with_tab_focus_data = []

  for user in list_users():
    if not sdir_exists('tab_focus_times_for_user/' + user + '.json'):
      continue
    tab_focus_times = get_tab_focus_times_for_user(user)
    ordered_visits = get_history_ordered_visits_for_user(user)
    start_time = tab_focus_times[0]['start']
    ordered_visits_after_start = [x for x in ordered_visits if x['visitTime'] > start_time]
    #print len(tab_focus_times)
    if len(tab_focus_times) > 2000 and len(ordered_visits_after_start) > 400:
      users_with_tab_focus_data.append(user)
  return users_with_tab_focus_data
'''
#print len(users_with_tab_focus_data)

"\ndef get_users_with_tab_focus_data():\n  users_with_tab_focus_data = []\n\n  for user in list_users():\n    if not sdir_exists('tab_focus_times_for_user/' + user + '.json'):\n      continue\n    tab_focus_times = get_tab_focus_times_for_user(user)\n    ordered_visits = get_history_ordered_visits_for_user(user)\n    start_time = tab_focus_times[0]['start']\n    ordered_visits_after_start = [x for x in ordered_visits if x['visitTime'] > start_time]\n    #print len(tab_focus_times)\n    if len(tab_focus_times) > 2000 and len(ordered_visits_after_start) > 400:\n      users_with_tab_focus_data.append(user)\n  return users_with_tab_focus_data\n"

In [34]:
#user = 'dwCCD1uofp' # doing very poorly with this user
#user = 'UrZvGUsI8N'
#user = users_with_tab_focus_data[0]
#print user
#url_to_visits = get_history_visits_for_user(user)

In [35]:
'''
def compute_tab_focus_times_with_events_for_user(user):
  current_session_tracker = SessionTracker(store_events=True)
  for line in get_log_with_mlog_active_times_for_user(user):
    current_session_tracker.process_input(uncompress_data_subfields(line))
  current_session_tracker.end_input()
  return current_session_tracker.get_output()
'''

'\ndef compute_tab_focus_times_with_events_for_user(user):\n  current_session_tracker = SessionTracker(store_events=True)\n  for line in get_log_with_mlog_active_times_for_user(user):\n    current_session_tracker.process_input(uncompress_data_subfields(line))\n  current_session_tracker.end_input()\n  return current_session_tracker.get_output()\n'

In [36]:
#tab_focus_times_with_events = compute_tab_focus_times_with_events_for_user(user)


In [37]:
'''
for line in get_log_with_mlog_active_times_for_user(user):
  data = uncompress_data_subfields(line)
  url = get_focused_tab(data)
  if url != None:
    print url
    print data
    break
'''

'\nfor line in get_log_with_mlog_active_times_for_user(user):\n  data = uncompress_data_subfields(line)\n  url = get_focused_tab(data)\n  if url != None:\n    print url\n    print data\n    break\n'

In [38]:
#print len(tab_focus_times_with_events)
#tab_focus_times_with_events[0]

In [39]:
#tab_focus_times = get_tab_focus_times_for_user(user)
#print len(tab_focus_times)
#print tab_focus_times[0]
#print tab_focus_times

In [40]:
#ordered_visits = get_history_ordered_visits_for_user(user)
#first_site = url_to_visits.keys()[0]
#print url_to_visits[first_site]

In [41]:
'''
ordered_visits_after_start = []
for visit in ordered_visits:
  time = visit['visitTime']
  #if 1458171960840.0 <= time <= 1458172165207.0:
  #  print visit
  if time >= 1458171960840.0:
    ordered_visits_after_start.append(visit)
'''

"\nordered_visits_after_start = []\nfor visit in ordered_visits:\n  time = visit['visitTime']\n  #if 1458171960840.0 <= time <= 1458172165207.0:\n  #  print visit\n  if time >= 1458171960840.0:\n    ordered_visits_after_start.append(visit)\n"

In [42]:
#print len(ordered_visits_after_start)

In [43]:
'''
def to_visit_lengths(ordered_visits):
  focus_time_reconstructor = ReconstructFocusTimesBaseline()
  for visit in ordered_visits:
    focus_time_reconstructor.process_history_line(visit)
  return focus_time_reconstructor.get_output()

reconstructed_visit_lengths = to_visit_lengths(ordered_visits)
'''

'\ndef to_visit_lengths(ordered_visits):\n  focus_time_reconstructor = ReconstructFocusTimesBaseline()\n  for visit in ordered_visits:\n    focus_time_reconstructor.process_history_line(visit)\n  return focus_time_reconstructor.get_output()\n\nreconstructed_visit_lengths = to_visit_lengths(ordered_visits)\n'

In [44]:
#print len(reconstructed_visit_lengths)

In [45]:
#print ref_start_time

In [46]:
#print evaluated_tab_focus_times[0]

In [47]:
'''
for x in reconstructed_visit_lengths:
  url = x['url']
  if 'tmi.stanford.edu' in url:
    print x
    break
'''

"\nfor x in reconstructed_visit_lengths:\n  url = x['url']\n  if 'tmi.stanford.edu' in url:\n    print x\n    break\n"

In [48]:
#print len(evaluated_reconstructed_visits)
#print len(evaluated_tab_focus_times)

In [49]:
#def 

In [50]:
#print evaluated_reconstructed_visits[0]
#print evaluated_tab_focus_times[2]
#print evaluated_reconstructed_visits[-1]
#print evaluated_tab_focus_times[-1]

In [51]:
#print len(result_reference.get_output())
#print len(result_reconstructed.get_output())

In [52]:
'''
for ref_answer,rec_answer in zip(result_reference.get_output(), result_reconstructed.get_output())[:1000]:
  ref_domain = 'None'
  rec_domain = 'None'
  if ref_answer != 0:
    ref_url = result_reference.get_url_for_index(ref_answer)
    ref_domain = url_to_domain(ref_url)
  if rec_answer != 0:
    rec_url = result_reconstructed.get_url_for_index(rec_answer)
    rec_domain = url_to_domain(rec_url)
  print ref_domain, rec_domain
'''

"\nfor ref_answer,rec_answer in zip(result_reference.get_output(), result_reconstructed.get_output())[:1000]:\n  ref_domain = 'None'\n  rec_domain = 'None'\n  if ref_answer != 0:\n    ref_url = result_reference.get_url_for_index(ref_answer)\n    ref_domain = url_to_domain(ref_url)\n  if rec_answer != 0:\n    rec_url = result_reconstructed.get_url_for_index(rec_answer)\n    rec_domain = url_to_domain(rec_url)\n  print ref_domain, rec_domain\n"

In [53]:
#for x in evaluated_reconstructed_visits:
#  print x

In [54]:
#for x in evaluated_tab_focus_times:
#  print x

In [55]:
#for x in reconstructed_visit_lengths:
#  url = x['url']
#  if url == 'https://www.mturk.com/mturk/continue?hitId=3ZY8KE4ISJ42CAV8NVJ18F1PI9SQVJ':
#    print x

In [56]:
#for x in tab_focus_times:
#  url = x['url']
#  if url == 'https://www.mturk.com/mturk/continue?hitId=3ZY8KE4ISJ42CAV8NVJ18F1PI9SQVJ':
#    print x

In [57]:

'''
def to_visit_lengths(ordered_visits):
  output = []
  for item,idx in ordered_visits:
    url = item['url']
    visit_time = item['visitTime'] # timestamp in milliseconds
    result = {'url': url, 'start': visit_time, 'active': visit_time}
    visit_duration = 30*1000 # 30 seconds in milliseconds
    if idx < len(ordered_visits):
      next_item = ordered_visits[idx+1]
      next_visit_time = next_item['visitTime']
      time_until_next_visit = next_visit_time - visit_time
      if visit_duration > time_until_next_visit: # more than 30 minutes until next visit
        
      visit_duration = min(visit_duration, time_until_next_visit)
    output.append({'url': url, 'start': visit_time, })
'''

"\ndef to_visit_lengths(ordered_visits):\n  output = []\n  for item,idx in ordered_visits:\n    url = item['url']\n    visit_time = item['visitTime'] # timestamp in milliseconds\n    result = {'url': url, 'start': visit_time, 'active': visit_time}\n    visit_duration = 30*1000 # 30 seconds in milliseconds\n    if idx < len(ordered_visits):\n      next_item = ordered_visits[idx+1]\n      next_visit_time = next_item['visitTime']\n      time_until_next_visit = next_visit_time - visit_time\n      if visit_duration > time_until_next_visit: # more than 30 minutes until next visit\n        \n      visit_duration = min(visit_duration, time_until_next_visit)\n    output.append({'url': url, 'start': visit_time, })\n"